In [2]:
! pip install psycopg2-binary

In [ ]:
import psycopg2
import sys


def connect_to_db():
    """Establishes a connection to the PostgreSQL database."""
    try:
        conn = psycopg2.connect(
            dbname="database-instance",
            user="testuje-sobie",
            password="",
            host="34.140.62.43",
            port="5432",  # Default PostgreSQL port
        )
        return conn
    except psycopg2.OperationalError as e:
        print(f"Unable to connect to the database: {e}")
        sys.exit(1)  # Exits the script if connection fails
    except Exception as e:
        print(f"An unexpected error occurred during connection: {e}")
        sys.exit(1)


def create_table_if_not_exists(conn):
    """Creates a sample table if it doesn't already exist."""
    create_table_query = """
    CREATE TABLE IF NOT EXISTS employees (
        id SERIAL PRIMARY KEY,
        first_name VARCHAR(50) NOT NULL,
        last_name VARCHAR(50) NOT NULL,
        email VARCHAR(100) UNIQUE,
        hire_date DATE DEFAULT CURRENT_DATE
    );
    """
    try:
        with conn.cursor() as cur:
            cur.execute(create_table_query)
            conn.commit()  # Commit the changes for table creation
        print("Table 'employees' checked/created successfully.")
    except psycopg2.Error as e:
        print(f"Error creating table: {e}")
        conn.rollback()  # Rollback in case of error
        sys.exit(1)


def insert_single_employee(conn, first_name, last_name, email):
    """Inserts a single employee record into the 'employees' table."""
    insert_query = """
    INSERT INTO employees (first_name, last_name, email)
    VALUES (%s, %s, %s) RETURNING id;
    """
    employee_id = None
    try:
        with conn.cursor() as cur:
            cur.execute(insert_query, (first_name, last_name, email))
            employee_id = cur.fetchone()[0]  # Get the ID of the newly inserted row
            conn.commit()  # Commit the transaction
            print(
                f"Employee '{first_name} {last_name}' inserted successfully with ID: {employee_id}."
            )
    except psycopg2.IntegrityError as e:  # Catch potential errors like duplicate email
        print(f"Integrity error inserting employee '{first_name} {last_name}': {e}")
        conn.rollback()  # Rollback the transaction
    except psycopg2.Error as e:
        print(f"Error inserting single employee '{first_name} {last_name}': {e}")
        conn.rollback()  # Rollback the transaction
    except Exception as e:
        print(f"An unexpected error occurred during single insert: {e}")
        conn.rollback()
    return employee_id


def insert_multiple_employees(conn, employee_data):
    """
    Inserts multiple employee records into the 'employees' table.
    employee_data should be a list of tuples, e.g.,
    [('Alice', 'Smith', 'alice.smith@example.com'), ('Bob', 'Johnson', 'bob.j@example.com')]
    """
    insert_query = """
    INSERT INTO employees (first_name, last_name, email)
    VALUES (%s, %s, %s);
    """
    try:
        with conn.cursor() as cur:
            cur.executemany(insert_query, employee_data)
            conn.commit()  # Commit the transaction
            print(f"Successfully inserted {cur.rowcount} employees.")
    except psycopg2.IntegrityError as e:
        print(f"Integrity error inserting multiple employees: {e}")
        conn.rollback()
    except psycopg2.Error as e:
        print(f"Error inserting multiple employees: {e}")
        conn.rollback()
    except Exception as e:
        print(f"An unexpected error occurred during multiple insert: {e}")
        conn.rollback()

In [4]:
if __name__ == "__main__":
    conn = None
    try:
        # Establish database connection
        conn = connect_to_db()

        # Ensure the table exists (optional, good for testing)
        create_table_if_not_exists(conn)

        # Example: Insert a single employee
        insert_single_employee(conn, "John", "Doe", "john.doe@example.com")
        insert_single_employee(
            conn, "Jane", "Doe", "jane.doe@example.com"
        )  # Another single insert

        # Example: Insert an employee that might cause an integrity error (if email is unique)
        insert_single_employee(conn, "John", "Dorian", "john.doe@example.com")

        # Example: Insert multiple employees
        employees_to_add = [
            ("Peter", "Pan", "peter.pan@example.com"),
            ("Wendy", "Darling", "wendy.darling@example.com"),
            ("Captain", "Hook", "hook@example.org"),
        ]
        insert_multiple_employees(conn, employees_to_add)

        # Example: Another batch, one of which might be a duplicate
        more_employees = [
            ("Tinker", "Bell", "tinker.bell@example.com"),
            ("Smee", "Pirate", "hook@example.org"),  # Potential duplicate email
        ]
        insert_multiple_employees(conn, more_employees)

    except Exception as e:
        print(f"A critical error occurred in the main execution: {e}")
    finally:
        if conn:
            conn.close()
            print("Database connection closed.")

Unable to connect to the database: connection to server at "34.140.62.43", port 5432 failed: Operation timed out
	Is the server running on that host and accepting TCP/IP connections?



AttributeError: 'tuple' object has no attribute 'tb_frame'